In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import cx_Oracle
import os
import pandas as pd

wallet_location = '/home/ubuntu/esports_wallet'

os.environ['TNS_ADMIN'] = wallet_location

!echo $TNS_ADMIN

/home/ubuntu/esports_wallet


In [2]:
import os

os.environ['PATH']='/home/ubuntu/miniconda3/bin:$PATH'

In [3]:
!python -m pip install -U pip
!python -m pip install -U setuptools wheel
!python -m pip install -U "mxnet<2.0.0"
!python -m pip install autogluon

!pip install dataprep
!pip install dask
!pip install pandas_profiling
## install packages
!pip install -q scikit-learn

  Using cached dataprep-0.4.1-py3-none-any.whl (3.5 MB)
  Using cached varname-0.8.1-py3-none-any.whl (20 kB)
  Using cached aiohttp-3.8.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Using cached levenshtein-0.12.0-cp37-cp37m-manylinux1_x86_64.whl (158 kB)
  Using cached Bottleneck-1.3.2.tar.gz (88 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Jinja2-2.11.3-py2.py3-none-any.whl (125 kB)
  Using cached ipywidgets-7.6.5-py2.py3-none-any.whl (121 kB)
  Using cached Metaphone-0.6-py3-none-any.whl
  Using cached bokeh-2.4.2-py3-none-any.whl (18.5 MB)
  Using cached wordcloud-1.8.1-cp37-cp37m-manylinux1_x86_64.whl (366 kB)
  Using cached Flask_Cors-3.0.10-py2.py3-none-any.whl (14 kB)
  Using cached nltk-3.6.5-py3-none-any.whl (1.5 MB)
  Using cached usaddress-0.5.10-py2.py3-none-any.whl (63 kB)
  Using cached dask-2.30.0-p

  Using cached frozenlist-1.2.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (192 kB)
  Using cached asynctest-0.13.0-py3-none-any.whl (26 kB)
  Using cached yarl-1.7.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (271 kB)
  Using cached async_timeout-4.0.1-py3-none-any.whl (5.7 kB)
  Using cached multidict-5.2.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (160 kB)
  Using cached aiosignal-1.2.0-py3-none-any.whl (8.2 kB)
  Using cached Werkzeug-1.0.1-py2.py3-none-any.whl (298 kB)
  Using cached itsdangerous-1.1.0-py2.py3-none-any.whl (16 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached jupyterlab_widgets-1.0.2-py3-none-any.whl (243 kB)
  Using cached widgetsnbextension-3.5.2-py2.py3-none-any.whl (1.6 MB)
  Using cached ply-3.11-py2.py3-none-any.whl (49 kB)
  Using cached nltk-3.6.3-py3-none-any.whl (1.5 MB

  ERROR: Command errored out with exit status 1:
   command: /home/ubuntu/miniconda3/bin/python /home/ubuntu/miniconda3/lib/python3.7/site-packages/pip/_vendor/pep517/in_process/_in_process.py build_wheel /tmp/tmp_yf65u2w
       cwd: /tmp/pip-install-y00if1l1/bottleneck_d3234a11cade430bb9ecc849c426e839
  Complete output (122 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/bottleneck
  copying bottleneck/_pytesttester.py -> build/lib.linux-x86_64-3.7/bottleneck
  copying bottleneck/__init__.py -> build/lib.linux-x86_64-3.7/bottleneck
  copying bottleneck/_version.py -> build/lib.linux-x86_64-3.7/bottleneck
  creating build/lib.linux-x86_64-3.7/bottleneck/tests
  copying bottleneck/tests/__init__.py -> build/lib.linux-x86_64-3.7/bottleneck/tests
  copying bottleneck/tests/list_input_test.py -> build/lib.linux-x86_64-3.7/bottleneck/tests
  copying bottleneck/tests/reduce_test.py -

In [4]:
import cx_Oracle
import yaml
import os
from pathlib import Path
home = str(Path.home())

def process_yaml():
	with open("../config.yaml") as file:
		return yaml.safe_load(file)


class OracleJSONDatabaseConnection:
    def __init__(self, data=process_yaml()):
        # wallet location (default is HOME/wallets/wallet_X)
        os.environ['TNS_ADMIN'] = '{}/{}'.format(home, process_yaml()['WALLET_DIR'])
        print(os.environ['TNS_ADMIN'])
        self.pool = cx_Oracle.SessionPool(data['db']['username'], data['db']['password'], data['db']['dsn'],
            min=1, max=4, increment=1, threaded=True,
            getmode=cx_Oracle.SPOOL_ATTRVAL_WAIT
        )
        print('Connection successful.')



    def close_pool(self):
        self.pool.close()
        print('Connection pool closed.')



    def insert(self, collection_name, json_object_to_insert):
        connection = self.pool.acquire()
        connection.autocommit = True
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)

        try:
            x_collection.insertOne(json_object_to_insert)
            print('[DBG] INSERT {} OK'.format(json_object_to_insert))
        except cx_Oracle.IntegrityError:
            print('[DBG] INSERT {} ERR'.format(json_object_to_insert))
            return 0
        self.pool.release(connection)
        return 1



    def delete(self, collection_name, on_column, on_value):
        connection = self.pool.acquire()
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)
        qbe = {on_column: on_value}
        x_collection.find().filter(qbe).remove()
        self.pool.release(connection)



    def get_connection(self):
        return self.pool.acquire() 



    def close_connection(self, conn_object):
        self.pool.release(conn_object)



    def get_collection_names(self):
        connection = self.pool.acquire()
        returning_object = connection.getSodaDatabase().getCollectionNames(startName=None, limit=0)
        self.pool.release(connection)
        return returning_object



    def open_collection(self, collection_name):
        connection = self.pool.acquire()
        returning_object = self.pool.acquire().getSodaDatabase().openCollection(collection_name)
        self.pool.release(connection)
        return returning_object



def test_class():
    object = OracleJSONDatabaseConnection()
    print(object.pool)
    object.close_pool()

In [5]:
print(os.environ['TNS_ADMIN'])

/home/ubuntu/esports_wallet


In [6]:
db = OracleJSONDatabaseConnection()
print(db.get_collection_names())

/home/ubuntu/wallets/Wallet_eSportsDB
Connection successful.
['1v1_model', 'match', 'match_detail', 'matchups', 'predictor', 'predictor_liveclient', 'summoner']


In [7]:
data = db.open_collection('predictor')
all_data = list()
i = 0
for doc in data.find().getCursor():
    content = doc.getContent()
    all_data.append(content)
    i+= 1
    if i > 1000000:
        break

print('Data length: {}'.format(len(all_data)))

Data length: 1000001


In [8]:
df = pd.read_json(json.dumps(all_data), orient='records')

df.head(5)


,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
0,0,100,1035,33,0,5,9,0,240047,0,...,6751,103,810,0,710,48,679,8388,3,0
1,0,0,1113,34,3765,5,0,0,240047,0,...,3894,107,61,0,834,46,1266,3894,4,0
2,0,0,1316,38,0,0,0,0,240047,0,...,5930,112,291,150,809,32,901,8044,3,0
3,0,0,1136,30,30887,0,0,0,240047,0,...,3355,118,414,0,640,39,901,3355,3,0
4,1,0,1565,36,13147,10,4,0,300069,0,...,6905,109,0,50,838,48,1816,7253,5,0


In [9]:
df.describe()

,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
count,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1000001.0,...,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1.000001e+06,1000001.0
mean,4.999465e-01,4.204545e+02,5.354515e+03,4.532781e+01,1.049577e+05,6.403167e+00,1.649709e+01,5.272095e-02,8.707893e+05,0.0,...,2.854528e+04,1.411292e+02,2.895108e+03,2.682114e+03,1.377490e+03,6.952831e+01,5.918912e+03,4.917680e+04,8.402085e+00,0.0
std,5.000002e-01,7.379546e+02,3.929124e+03,2.150004e+01,1.441895e+05,1.235793e+01,3.513823e+01,9.777257e-01,5.647913e+05,0.0,...,4.048491e+04,4.468171e+01,3.480716e+03,4.785019e+03,6.210390e+02,3.908916e+01,4.643858e+03,5.332150e+04,4.428634e+00,0.0
min,0.000000e+00,0.000000e+00,5.000000e+02,-1.000000e+01,0.000000e+00,-3.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,0.000000e+00,4.700000e+01,0.000000e+00,0.000000e+00,3.400000e+02,-1.400000e+01,0.000000e+00,0.000000e+00,1.000000e+00,0.0
25%,0.000000e+00,0.000000e+00,2.056000e+03,3.300000e+01,1.155800e+04,0.000000e+00,0.000000e+00,0.000000e+00,4.201010e+05,0.0,...,2.993000e+03,1.120000e+02,3.890000e+02,7.900000e+01,8.920000e+02,4.500000e+01,2.026000e+03,8.589000e+03,5.000000e+00,0.0
50%,0.000000e+00,1.500000e+02,4.688000e+03,3.800000e+01,5.574800e+04,0.000000e+00,0.000000e+00,0.000000e+00,8.402370e+05,0.0,...,1.006200e+04,1.250000e+02,1.668000e+03,7.910000e+02,1.293000e+03,5.900000e+01,5.164000e+03,2.921600e+04,9.000000e+00,0.0
75%,1.000000e+00,5.120000e+02,7.877000e+03,4.700000e+01,1.408080e+05,7.000000e+00,1.000000e+01,0.000000e+00,1.260471e+06,0.0,...,3.861200e+04,1.540000e+02,4.198000e+03,3.033000e+03,1.726000e+03,8.200000e+01,8.986000e+03,7.593800e+04,1.200000e+01,0.0
max,1.000000e+00,2.153000e+04,2.887900e+04,5.190000e+02,3.070076e+06,8.400000e+01,2.780000e+02,6.000000e+01,3.147362e+06,0.0,...,4.691170e+05,6.990000e+02,4.768900e+04,8.332400e+04,7.534000e+03,1.308000e+03,3.382200e+04,5.418970e+05,1.800000e+01,0.0


In [10]:
from pandas_profiling import ProfileReport

In [11]:
report = ProfileReport(df)
#report #uncomment to display all.

In [12]:
from autogluon.tabular import TabularPredictor, TabularDataset

In [13]:
df = TabularDataset(df)

# drop columns we don't want
df = df.drop(columns=['identifier', 'participantId', 'timestamp'])

train = df.sample(frac=0.8,random_state=200) #random state is a seed value
test = df.drop(train.index)

train.head(5)

,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,bonusArmorPenPercent,magicPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
240991,0,985,11880,38,4241,0,4,0,0,0,...,109219,173,4318,1859,1797,97,13051,126526,15,0
619620,1,532,8855,35,75536,0,0,0,0,0,...,3195,116,8372,7125,2020,72,10965,27840,13,0
315491,1,0,727,30,11294,0,0,0,0,0,...,6,102,14,56,560,36,550,658,2,0
491785,0,802,8096,48,191486,0,8,0,0,0,...,12590,136,2279,7293,1903,75,11275,110372,13,0
230307,1,1488,9651,45,183028,30,119,0,0,0,...,103667,129,8331,1387,1915,80,10856,124120,13,0


In [14]:
label = 'winner'

In [15]:
save_path = '/home/ubuntu/models/autogluon_trained_models'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train, time_limit=600)

Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "/home/ubuntu/models/autogluon_trained_models/"
AutoGluon Version:  0.3.1
Train Data Rows:    800001
Train Data Columns: 41
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    254583.84 MB
	Train Data (Original)  Memory Usage: 262.4 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsT

In [16]:
y_test = test[label]  # values to predict
test_data_nolabel = test.drop(columns=[label])  # delete label column to prove we're not cheating, also drop identifier column
test_data_nolabel.head(5)

,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,bonusArmorPenPercent,magicPenPercent,armorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
0,100,1035,33,0,5,9,0,0,0,0,...,6751,103,810,0,710,48,679,8388,3,0
2,0,1316,38,0,0,0,0,0,0,0,...,5930,112,291,150,809,32,901,8044,3,0
8,0,923,33,0,25,0,0,0,0,0,...,3517,116,0,82,762,50,904,5388,3,0
9,0,1113,33,0,0,20,0,0,0,0,...,8665,108,469,0,710,48,985,10533,3,0
13,0,4423,40,14009,15,0,0,0,0,0,...,19851,185,6,1226,1556,91,5952,45212,9,0


In [17]:
predictor = TabularPredictor.load(save_path)

y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


Predictions:  
 0          0
2          0
8          1
9          1
13         1
          ..
999989     1
999994     0
999996     1
999999     0
1000000    1
Name: winner, Length: 200000, dtype: int64


Evaluation: accuracy on test data: 0.741655
Evaluations on test data:
{
    "accuracy": 0.741655,
    "balanced_accuracy": 0.7416342038661585,
    "mcc": 0.48330473492130893,
    "f1": 0.7394914767141108,
    "precision": 0.7428811653514592,
    "recall": 0.7361325811567726
}


In [18]:
predictor.leaderboard(test, silent=False)

                 model  score_test  score_val  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2    0.741655   0.746907        9.642585       0.654524  158.604869                 0.006366                0.011860           1.196267            2       True          7
1     RandomForestGini    0.740370   0.742282        4.527414       0.319459   71.960550                 4.527414                0.319459          71.960550            1       True          5
2     RandomForestEntr    0.738965   0.739533        5.108804       0.323206   85.448052                 5.108804                0.323206          85.448052            1       True          6
3           LightGBMXT    0.620185   0.624172        0.292148       0.029704    9.360344                 0.292148                0.029704           9.360344            1       True          3
4       KNeighborsDist    0.617570   0.6

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.741655,0.746907,9.642585,0.654524,158.604869,0.006366,0.011860,1.196267,2,True,7
1,RandomForestGini,0.740370,0.742282,4.527414,0.319459,71.960550,4.527414,0.319459,71.960550,1,True,5
2,RandomForestEntr,0.738965,0.739533,5.108804,0.323206,85.448052,5.108804,0.323206,85.448052,1,True,6
3,LightGBMXT,0.620185,0.624172,0.292148,0.029704,9.360344,0.292148,0.029704,9.360344,1,True,3
4,KNeighborsDist,0.617570,0.615298,4201.254280,167.863369,0.717816,4201.254280,167.863369,0.717816,1,True,2
5,LightGBM,0.603740,0.606799,0.056793,0.019441,2.299721,0.056793,0.019441,2.299721,1,True,4
6,KNeighborsUnif,0.593600,0.589801,4209.265886,168.717546,0.716202,4209.265886,168.717546,0.716202,1,True,1


In [19]:
predictor.feature_importance(test)

Computing feature importance via permutation shuffling for 41 features using 1000 rows with 3 shuffle sets...
	582.54s	= Expected runtime (194.18s per shuffle set)
	39.39s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
totalGold,0.121000,0.019079,0.004093,3,0.230324,0.011676
totalDamageTaken,0.110000,0.010149,0.001413,3,0.168154,0.051846
magicDamageTaken,0.078667,0.010214,0.002786,3,0.137196,0.020137
physicalDamageTaken,0.048667,0.001528,0.000164,3,0.057420,0.039914
trueDamageTaken,0.045333,0.001528,0.000189,3,0.054086,0.036580
minionsKilled,0.034333,0.005774,0.004647,3,0.067416,0.001251
abilityPower,0.030000,0.007937,0.011274,3,0.075481,-0.015481
armor,0.028000,0.009165,0.016954,3,0.080517,-0.024517
totalDamageDoneToChampions,0.026333,0.015308,0.048298,3,0.114050,-0.061383
magicResist,0.021333,0.005033,0.009027,3,0.050174,-0.007508


In [20]:
print('Finished')

Finished
